# UK Covid-19 Data


In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

from datetime import timedelta
from tqdm.notebook import trange, tqdm
from tsmoothie.smoother import LowessSmoother
from uk_covid19 import Cov19API

## Get data from the server
https://coronavirus.data.gov.uk/developers-guide

Please be patient, this may take some time (2-3 minutes)...

In [2]:
filters = ['areaType=ltla']

structure = {
    'date': 'date',
    'areaName': 'areaName',
    'areaCode': 'areaCode',
    'newCasesBySpecimenDate': 'newCasesBySpecimenDate',
    'cumCasesBySpecimenDate': 'cumCasesBySpecimenDate'
}

api = Cov19API(filters=filters, structure=structure)
df = api.get_dataframe()

# Correct the column types
df['newCasesBySpecimenDate'] = df['newCasesBySpecimenDate'].astype(float)
df['cumCasesBySpecimenDate'] = df['cumCasesBySpecimenDate'].astype(float)

# Sort values by area code and date
df.sort_values(by=['areaCode','date'], inplace=True)

## Apply smoothing to the new cases
The daily data tends to be quite noisy, therefore, apply lowess smoothing.
Note: some data is missing which trips up the smoothing function.

In [3]:
# Full missing values with zero
df['newCasesBySpecimenDate'] = df['newCasesBySpecimenDate'].apply(lambda x: x if pd.notnull(x) else 0)

lowess_smoother = LowessSmoother(smooth_fraction=0.05, iterations=1)

df['new cases'] = 0.0
ltlas = df[['areaCode']].groupby(by=['areaCode'], as_index=False).sum().values.tolist()
for ltla in tqdm(ltlas):
    df.loc[df['areaCode'].isin(ltla), ['new cases']] =  lowess_smoother.smooth(df[df['areaCode'].isin(ltla)]['newCasesBySpecimenDate']).smooth_data.T

## Add population
Population downloaded from NOMIS for the 2019 populations at Lower Tier Local Authority level.

In [4]:
population = pd.read_excel('Population/LTLA2019.xlsx')

## Chart Dorset data
BCP = E06000058

Dorset = E06000059


In [5]:
# dorset = df_b[df_b['areaCode'].isin(['E06000058', 'E06000059'])]
dorset = df[df['areaCode'].isin(['E06000058'])]
plt = px.line(dorset, x='date', y='new cases', title='Bournemouth, Christchurch, Poole', width=1280, height=540)
plt.add_trace(px.bar(dorset, x='date', y='newCasesBySpecimenDate', opacity=0.4, width=1280, height=540).data[0])
plt.show()

In [6]:
# dorset = df_b[df_b['areaCode'].isin(['E06000058', 'E06000059'])]
dorset = df[df['areaCode'].isin(['E06000059'])]
plt = px.line(dorset, x='date', y='new cases', title='Dorset', width=1280, height=540)
plt.add_trace(px.bar(dorset, x='date', y='newCasesBySpecimenDate', opacity=0.4, width=1280, height=540).data[0])
plt.show()

## National data and outlier detection

In [7]:
merged = pd.merge(df, population, how='left', left_on='areaCode', right_on='AreaCode')

In [8]:
merged['CasePer100k'] = merged['new cases'] / merged['Population2019'] * 100000

In [9]:
plt = px.box(merged, x='date', y='CasePer100k', width=1280, height=540)
dorset = pd.pivot_table(merged[merged['areaCode'].isin(['E06000059', 'E06000058'])], index='date', columns='areaName', values='CasePer100k', aggfunc=np.mean)
dorset.reset_index(inplace=True)
df2 = merged[merged['areaCode'].isin(['E06000058'])]
plt.add_trace(px.line(df2, x='date', y='CasePer100k', color_discrete_sequence=['red', 'yellow'], width=1280, height=540).data[0])
df2 = merged[merged['areaCode'].isin(['E06000059'])]
plt.add_trace(px.line(df2, x='date', y='CasePer100k', color_discrete_sequence=['yellow', 'yellow'], width=1280, height=540).data[0])
plt.show()

In [10]:
# Calculate median, quartiles and box plot thresholds
lq = merged[['date', 'CasePer100k']].groupby('date', as_index=False).quantile(q=0.25).rename(columns={'CasePer100k': 'lq'})
median = merged[['date', 'CasePer100k']].groupby('date', as_index=False).quantile(q=0.5).rename(columns={'CasePer100k': 'median'})
uq = merged[['date', 'CasePer100k']].groupby('date', as_index=False).quantile(q=0.75).rename(columns={'CasePer100k': 'uq'})
stats = pd.concat([lq, median['median'], uq['uq']], axis=1)
stats['iqr'] = stats['uq'] - stats['lq']
stats['L1'] = stats['uq'] + 1.5 * stats['iqr']
stats['L2'] = stats['uq'] + 3.0 * stats['iqr']

In [11]:
merged = pd.merge(merged, stats, how='left', left_on='date', right_on='date')

In [12]:
merged['outlierL1'] = merged['CasePer100k'] > merged['L1']
merged['outlierL2'] = merged['CasePer100k'] > merged['L2']

In [13]:
# Calculate outliers 2 days before the end of the data series (to avoid missing and incomplete data)
outliersL2 = merged[(pd.to_datetime(merged['date']) == (pd.to_datetime(merged['date'].max()) - timedelta(days=2))) & (merged['outlierL2'])][['areaCode', 'areaName']]

In [14]:
outliersL2

,areaCode,areaName
1167,E06000006,Halton
1557,E06000008,Blackburn with Darwen
27560,E07000117,Burnley
28150,E07000120,Hyndburn
28528,E07000122,Pendle
28722,E07000123,Preston
29102,E07000125,Rossendale
49070,E08000001,Bolton
49277,E08000002,Bury
49480,E08000003,Manchester
